# Notebook 1: Basic Example

Here we will introduce a basic example of how **BATTLESIMULATOR** works and can be of benefit to you.

### Requirements:

- `numpy`
- `pandas`
- `matplotlib`
- `itertools`
- `scipy.stats`
- `numba`

In [1]:
import sys
# map path back to the directory above battlesim/
sys.path.insert(0,"../")
# our main import
import battlesim as bsm

### Other Imports

In [2]:
from IPython.display import HTML
%matplotlib inline

## Create a Battle using the CloneWars Dataset

In [3]:
battle = bsm.Battle("../datasets/starwars-clonewars.csv")

### Within the `db_` object, we can see the table of information

This table represents the *knowledge* or statistics of each unit. These can be your own values/unit names, but the following column names **MUST** be present and the same within your own file, ideally in `.csv` file format.

In [4]:
battle.db_

,Allegiance,Type,Armor,HP,Damage,Dmg Speed,Range,Movement Speed,Accuracy,Miss,Shield,Shield Regen,allegiance_int
Name,,,,,,,,,,,,,
local militia,Republic,Standard,15,20,10,1.00,2.0,0.6,5,30,0,0.00,0
b1 battledroid,CIS,Standard,35,20,15,1.00,2.5,0.5,30,35,0,0.00,1
clone trooper,Republic,Standard,50,35,16,1.20,3.0,0.7,65,75,0,0.00,0
b2 battledroid,CIS,Standard,100,40,20,1.20,3.5,0.4,70,40,0,0.00,1
arc trooper,Republic,Elite,100,100,20,0.90,5.0,1.0,90,95,0,0.00,0
bx-series droid commando,CIS,Elite,70,50,22,1.30,7.0,0.8,70,95,0,0.00,1
clone sharpshooter,Republic,Specialist,50,40,50,0.20,15.0,0.2,60,40,0,0.00,0
battle droid assassin,CIS,Specialist,60,20,50,0.15,15.0,0.2,55,45,0,0.00,1
clone commando,Republic,Elite,100,120,20,1.50,3.0,0.9,97,97,30,0.20,0


## Initialise

The other elements within `battle` are not initialized yet, we need to *declare armies* using the names on the columns above, followed by the number of that unit we wish to use.

More than one **army** can be created in a function call. *Case is ignored*.

`create_army()` returns the `Battle` object to allow for function-chaining.

Multiple calls to `create_army()` override the previous calls, so armies are not *stacked*.

In [5]:
battle.create_army([("b1 battledroid", 10), ("clone trooper", 5)])

bsm.Battle(init=True, n_armies=2, simulated=False)

### We can view the armies we have stored:

In [6]:
battle.army_set_

[('b1 battledroid', 10), ('clone trooper', 5)]

### We can view the composition:

By default, our units are set a *random* initial and rolling AI to select targets from. This can be overrided as shown later.

In [7]:
battle.composition_

,unit,allegiance,n,init_ai,rolling_ai
0,b1 battledroid,CIS,10,random,random
1,clone trooper,Republic,5,random,random


### Internally, a matrix `M` is created:

This enables efficiency when we call simulation functions, to have it within **numpy arrays**.

In [8]:
battle.M_

array([(1, 0, [0.21836662, 0.53392083], 20., 2.5, 0.5, 0.3 , 0.35, 15., 13),
       (1, 0, [0.82741711, 0.62338406], 20., 2.5, 0.5, 0.3 , 0.35, 15., 11),
       (1, 0, [0.44581798, 0.64205481], 20., 2.5, 0.5, 0.3 , 0.35, 15., 13),
       (1, 0, [0.66180024, 0.89435674], 20., 2.5, 0.5, 0.3 , 0.35, 15., 13),
       (1, 0, [0.47216876, 0.8568431 ], 20., 2.5, 0.5, 0.3 , 0.35, 15., 13),
       (1, 0, [0.39939455, 0.00333016], 20., 2.5, 0.5, 0.3 , 0.35, 15., 14),
       (1, 0, [0.0567025 , 0.63757203], 20., 2.5, 0.5, 0.3 , 0.35, 15., 13),
       (1, 0, [0.07562349, 0.27677566], 20., 2.5, 0.5, 0.3 , 0.35, 15., 12),
       (1, 0, [0.4536283 , 0.24725454], 20., 2.5, 0.5, 0.3 , 0.35, 15., 14),
       (1, 0, [0.00499243, 0.96519882], 20., 2.5, 0.5, 0.3 , 0.35, 15., 11),
       (0, 1, [0.97353119, 0.32095963], 35., 3. , 0.7, 0.65, 0.75, 16.,  0),
       (0, 1, [0.9224803 , 0.43577652], 35., 3. , 0.7, 0.65, 0.75, 16.,  4),
       (0, 1, [0.47518084, 0.81067404], 35., 3. , 0.7, 0.65, 0.75, 16.,  0),

## Changing Attributes

Right now, despite the fact that we have two *army groups* who will attack each other, they currently sit on top of each other as they are randomly distributed around $(0, 0)$. 

To change this, we need to *apply positioning* to each army group. There are several ways to do this:

1. Create `Distribution` objects for each army group. A Distribution is based on a statistical distribution such as normal, uniform or other, with given parameters.
2. Define distributions using the parameters found in certain `scipy.stats.*` distributions.
3. Use a premade *distribution function* within the `Battle` object and simply pass the statistical parameters desired. Note that these functions are **deprecated**.

Multiple calls to `apply_position_*` will **override** previous calls.

I personally prefer passing dictionaries for each army group, giving the name/id/dist name, with associated parameters. You can also specify to separate $x$ and $y$ parameters using `x_` or `y_` as prefixes to the parameter you wish to adjust. These can be suffixes instead.

In [9]:
battle.apply_position([
    dict(name="gaussian", x_loc=0, y_loc=0, scale=1),
    dict(name="gaussian", x_loc=10, y_loc=5, scale=1)
])

bsm.Battle(init=True, n_armies=2, simulated=False)

Note that `battle.apply_position_gaussian` and `battle.apply_position_uniform` have deprecated and no longer in development.

### Changing AI behaviour

By default, every army group will initialise by choosing a random enemy. Once that enemy is dead, it will then randomly select a new enemy and move towards it.

This can be altered using a number of AI functions, these include:

- `random`: selects a random enemy.
- `nearest`: selects the closest enemy.
- `pack`: selects the most common enemy among your closest allies.
- `weakest`: selects the enemy with the lowest HP.
- `strongest`: selects the enemy with the highest HP.

This can be chosen at the initial stage before the simulation starts (`init_ai`) or during the running of the simulation (`rolling_ai`).

Each element in the list corresponds to that army groups AI programming. It effects every unit within that army group.

In [10]:
battle.set_initial_ai(["random", "nearest"])

bsm.Battle(init=True, n_armies=2, simulated=False)

In [12]:
battle.set_rolling_ai(["nearest", "close_weak"])

bsm.Battle(init=True, n_armies=2, simulated=False)

#### Checking the composition again...

We see that the AIs have changed.

In [13]:
battle.composition_

,unit,allegiance,n,init_ai,rolling_ai
0,b1 battledroid,CIS,10,nearest,nearest
1,clone trooper,Republic,5,close_weak,close_weak


## Running the Simulation

Simulations are run individually using the `simulate()` function. This function by default requires no parameters, but you can send some if you wish to influence some global attribute regarding the simulation.

Simulations can be called directly using the `M_` attribute and functions within the `simulator.py` file, such as `simulate_battle`.

By default, the `simulate()` function returns a `pandas.DataFrame` object which details the change of units over time. This should be stored in some object. **WARNING**: Large numbers of units within the simulation may cause this function to run slowly.

In addition, the `battle` object also stores an attribute called `sim_` which is the same as the object returned from the `simulate()` function, if you wish to refer to it later.

In [14]:
S = battle.simulate()

### The Results

The resulting object always contains *at least* the following columns:

- `frame`: Which frame of the simulation you are in, going from 0 to N.
- `army`: Which army the unit belongs to
- `allegiance`: Which team the unit belongs to
- `alive`: A boolean flag that indicates whether this unit is alive
- `x`, `y`: X-Y coordinates
- `dir_x`, `dir_y`: The direction the unit is pointing from $(0, 0)$ origin.

In [15]:
S.head()

,frame,army,allegiance,alive,x,y,dir_x,dir_y
0,0,0,1,True,-0.288737,0.920496,0.938067,0.346453
1,0,0,1,True,0.598970,1.412878,0.922449,0.386119
2,0,0,1,True,-0.686453,-0.123992,0.888274,0.459314
3,0,0,1,True,0.679490,0.324848,0.846073,0.533067
4,0,0,1,True,-0.011745,1.262795,0.943849,0.330377


## Visualization

One of the principle aims of this project was to actively visualise the simulation in 2D space over time. To do this, we provide a number of simulation plotting functions which you can directly use with ease. The easiest way to do this, within a *Jupyter Notebook*, is to call the convenience function `sim_jupyter()`. This function uses the cached `sim_` object (i.e the last fight simulated) and draws the variables using the data stored in the `db_` object.

In [16]:
battle.sim_jupyter()

Depending on your system and which packages you have installed on your system, you may need to fall back on to a more verbose way of generating the animation.

In [17]:
bsm.quiver_fight(S, allegiance_label=battle.allegiances_, 
                 allegiance_color={0:"red",1:"blue"})

### Generating HTML

The returned object from this function is from `matplotlib.animation.FuncAnimation`, an animation object. It contains a crucial function that converts this data into HTML/Javascript such that it can be displayed in Jupyter Notebook, called `to_jshtml()`.

In [18]:
html = bsm.quiver_fight(S).to_jshtml()

In [19]:
html[:200]

'\n<link rel="stylesheet"\nhref="https://maxcdn.bootstrapcdn.com/font-awesome/4.4.0/\ncss/font-awesome.min.css">\n<script language="javascript">\n  function isInternetExplorer() {\n    ua = navigator.userAge'

### Displaying the HTML

IPython display then provides us with a `HTML()` function which converts HTML code text into HTML output.

Widgets at the bottom provide interaction to play, pause, rewind etc.

### Explaining the GUI

Notice that every **unit** is represented as an arrow, with the colours assigned to each **allegiance**, when a unit dies it becomes a static cross (`x`). The arrows move towards each other until in range, then attack.

In [20]:
HTML(html)

### Extra Display Options

You may notice that random colours and labels have been assigned to **each army group**. This is because the meta-information from our `battle` object which contain labels like `[CIS, Republic]` are not passed to the drawing function. These have to be manually passed within our `quiver_fight()` function.

The function accepts a dictionary where the index is a number, referring to the army group, and the value is the text/color to display:

In [21]:
HTML(bsm.quiver_fight(S, {0:"Republic",1:"CIS"}, {0:'red', 1:'blue'}).to_jshtml())

There's the end of this first basic introduction to **BATTLESIMULATOR**.